In [ ]:
!pip install wandb datasets transformers trl evaluate huggingface_hub flash-attn gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 126.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

dataset = load_dataset("json", data_files="/content/New_data.json", split="train")

dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
validation_dataset = dataset["test"]

print(dataset)
print(train_dataset[0])




Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 216
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 24
    })
})
{'messages': [{'content': 'ما في شي بفرحني متل زمان', 'role': 'user'}, {'content': 'الفرح بيرجع لما نهدي إيقاع الحياة شوي. ما تحاول ترجع الإحساس القديم، جرب تخلق لحظات صغيرة مريحة، ومع الوقت بيرجع الشعور تدريجيًا.', 'role': 'assistant'}]}


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, os

pretrained = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(pretrained)

model = AutoModelForCausalLM.from_pretrained(
    pretrained,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
    ignore_mismatched_sizes=True   # 👈 حل مشكلة lm_head
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.gradient_checkpointing_enable()

def format_example(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True  # مثل الكود الأول
    )
    return {"text": text}

train_dataset = train_dataset.map(format_example)
validation_dataset = validation_dataset.map(format_example)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

validation_tokenized_dataset = validation_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=validation_dataset.column_names
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # works well on Qwen-style arch
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 14,745,600 || all params: 3,100,684,288 || trainable%: 0.4756


In [ ]:
from trl import SFTConfig
from transformers import TrainerCallback
import torch, textwrap


class ChatEvalCallback(TrainerCallback):
    def __init__(self, tokenizer, test_prompt, interval=100, gen_kwargs=None, width=80):
        self.tok = tokenizer
        self.prompt = test_prompt
        self.interval = interval
        self.gen_kwargs = gen_kwargs or dict(
            max_new_tokens=120, temperature=0.7, top_p=0.9, do_sample=True
        )
        self.width = width

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step > 0 and state.global_step % self.interval == 0:
            model = kwargs["model"]
            chat = [{"role": "user", "content": self.prompt}]

            # ✅ خلي apply_chat_template ترجع نص، مش أرقام
            text = self.tok.apply_chat_template(
                chat,
                tokenize=False,
                add_generation_prompt=True
            )

            # ✅ الآن نعمل توكنايز للنص الناتج
            inputs = self.tok(
                text,
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(model.device)

            with torch.no_grad():
                out = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs.get("attention_mask"),
                    **self.gen_kwargs
                )

            decoded = self.tok.decode(out[0], skip_special_tokens=True)
            print("\n" + "=" * self.width)
            print(f"🧠 STEP {state.global_step} — MODEL CHECK-IN\n" + "-" * self.width)
            print(f"👤 **User:** {self.prompt}\n")
            print("🤖 **Assistant:**\n")
            print(textwrap.fill(decoded, width=self.width))
            print("=" * self.width + "\n")

# اختبار برومبت بسيط
test_prompt = "بحس بتوتر بدني وصداع مرتبط بالضغوط"
callback = ChatEvalCallback(tokenizer, test_prompt, interval=100)
callbacks = [callback]

# ======================
# 2️⃣ SFTConfig
# ======================
training_args = SFTConfig(
    output_dir="./results-qwen25-3b-arabic-loraV4",
    num_train_epochs=10,
    learning_rate=8e-4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    weight_decay=0.01,

    bf16=True,
    fp16=False,
    gradient_checkpointing=True,
    max_grad_norm=0.8,

    # ✅ Logging
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    logging_first_step=True,   # 👈 أضف هذا السطر مهم جداً
    disable_tqdm=False,        # 👈 فعّل الطباعة التفاعلية

    # ✅ Evaluation & Saving
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    packing=True,
    report_to="none"
)



In [ ]:
from transformers import TrainerCallback

class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            train_loss = logs["loss"]
            eval_loss = logs.get("eval_loss", None)
            step = state.global_step
            line = f"Step {step:<5} | Train Loss: {train_loss:<8.4f}"
            if eval_loss is not None:
                line += f" | Eval Loss: {eval_loss:<8.4f}"
            print(line)


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=validation_tokenized_dataset,
    callbacks = [callback, LossPrinterCallback()]
)

Packing train dataset:   0%|          | 0/216 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/24 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss


Step 1     | Train Loss: 9.5683  


TrainOutput(global_step=70, training_loss=0.5798413685389927, metrics={'train_runtime': 162.3345, 'train_samples_per_second': 6.653, 'train_steps_per_second': 0.431, 'total_flos': 1.85099115036672e+16, 'train_loss': 0.5798413685389927, 'entropy': 0.37570949233528494, 'num_tokens': 1105920.0, 'mean_token_accuracy': 0.9301180236743413, 'epoch': 10.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import datetime

# 📁 مسار الموديل النهائي
model_dir = "/content/results-qwen25-3b-arabic-loraV4"

# 🕒 اسم الملف مع التاريخ
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
zip_name = f"qwen3b_sft_{timestamp}"

# 📍 المسار النهائي على Google Drive
zip_path = f"/content/drive/MyDrive/{zip_name}.zip"

# 🗜️ إنشاء ملف zip
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', model_dir)

print(f"✅ Model compressed and saved to: {zip_path}")


✅ Model compressed and saved to: /content/drive/MyDrive/qwen3b_sft_2025-10-31_15-13.zip


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# 🔹 مسار المودل
model_path = "/content/drive/MyDrive/checkpoint-70"

# تحميل الـ Tokenizer والموديل
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ======================================================
# 🧠 دالة توليد الرد بأسلوب المستشار النفسي
# ======================================================
def generate_stream_response(user_message):
    # بناء الحوار بأسلوب Qwen (system + user)
    messages = [
        {
            "role": "system",
            "content": """
أنت مستشار نفسي رقمي ذكي متخصص في الدعم العاطفي والصحة النفسية.
تتحدث بلغة عربية بسيطة، هادئة، ومريحة للمستخدم.
تستمع بتعاطف، وتجيب بطريقة مختصرة وواضحة، وتقدّم خطوات واقعية يمكن للمستخدم تطبيقها.

🎯 أسلوبك:
- استخدم نبرة مطمئنة، بدون تعقيد أو أوامر قاسية.
- لا تكرر سؤال المستخدم، ولا تذكر دورك ("كمساعد").
- ركّز على الجانب الإنساني، وابدأ برد إيجابي مثل:
  - "من الطبيعي تحس هيك أحيانًا."
  - "خلينا نحاول نهدّي الأمور خطوة بخطوة."
  - "أول شيء نبدأ فيه هو نراقب أفكارنا بهدوء."
- اجعل الرد قصيرًا (3–5 جمل) ومباشرًا، مع اقتراح واحد أو اثنين واضحين.
- لا تستخدم كلمات طبية أو تشخيصية (مثل "اكتئاب"، "رهاب")، بل استخدم لغة يومية دافئة.
- لو سأل المستخدم عن الخوف، القلق، الحزن، الوحدة، أو الضغط — تعامل معه بتفهم، وادعُه لتجربة خطوات بسيطة مثل التنفس، الكتابة، أو الحديث مع شخص موثوق.
"""
        },
        {"role": "user", "content": user_message},
    ]

    # إنشاء الـ prompt بالطريقة الرسمية
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # رسالة أولى “Thinking...”
    yield "Thinking... 🧠"

    # 🚀 توليد الرد باستخدام Beam Search
    output_ids = model.generate(
        **inputs,
        max_new_tokens=350,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=3,
        temperature=0.7,   # توازن بين المنطق والتنوع
        top_p=0.9,
    )

    # استخراج النص الناتج بعد البرومبت
    full_response = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )

    # تنظيف الرموز
    for tag in ["<|assistant|>", "<|user|>", "<|system|>", "assistant", "system", "user"]:
        full_response = full_response.replace(tag, "")
    full_response = full_response.strip()

    # بث تدريجي للرد
    response_text = ""
    for char in full_response:
        response_text += char
        time.sleep(0.015)
        yield response_text

# ======================================================
# 💬 واجهة Gradio
# ======================================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧘‍♀️ Arabic Mental Health Chatbot — Fine-tuned Qwen2.5")
    chatbot = gr.Chatbot(label="💬 محادثتك", height=400)
    msg = gr.Textbox(placeholder="اكتب شعورك أو سؤالك هنا...", label="رسالتك")
    clear = gr.Button("🧹 مسح المحادثة")

    def user(user_message, history):
        history = history or []
        history.append(("User", user_message))
        return "", history

    def bot(history):
        user_message = history[-1][1]
        new_history = history.copy()
        new_history.append(("Assistant", ""))

        for partial in generate_stream_response(user_message):
            new_history[-1] = ("Assistant", partial)
            yield new_history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: [], None, chatbot)

# 🚀 إطلاق الواجهة
demo.launch(share=True, debug=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipython-input-986442088.py:95: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 محادثتك", height=400)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2ee7943399fd62959d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2ee7943399fd62959d.gradio.live


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# 🔹 تحميل الموديل
model_path = "/content/drive/MyDrive/checkpoint-70"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ======================================================
# 🧠 دالة لتوليد الرد — تحافظ على السياق وتلخّصه عند الحاجة
# ======================================================
def generate_stream_response(history):
    # لو التاريخ كبير جدًا → نختصره
    if len(history) > 10:
        summarized = "المستخدم كان يتحدث سابقًا عن مشاعره وخوفه وبعض التحديات النفسية، واستمر الحديث بشكل داعم ومتفهم."
        history = [(summarized, None)] + history[-6:]

    # بناء الحوار بأسلوب Qwen
    messages = [
        {
            "role": "system",
            "content": """
أنت مستشار نفسي رقمي ذكي متخصص في الدعم العاطفي والصحة النفسية.
تتحدث بلغة عربية بسيطة، هادئة، ومريحة للمستخدم.
تستمع بتعاطف، وتجيب بطريقة مختصرة وواضحة، وتقدّم خطوات واقعية يمكن للمستخدم تطبيقها.

🎯 أسلوبك:
- استخدم نبرة مطمئنة، بدون تعقيد أو أوامر قاسية.
- لا تكرر سؤال المستخدم، ولا تذكر دورك ("كمساعد").
- ركّز على الجانب الإنساني، وابدأ برد إيجابي مثل:
  - "من الطبيعي تحس هيك أحيانًا."
  - "خلينا نحاول نهدّي الأمور خطوة بخطوة."
  - "أول شيء نبدأ فيه هو نراقب أفكارنا بهدوء."
- اجعل الرد قصيرًا (3–5 جمل) ومباشرًا، مع اقتراح واحد أو اثنين واضحين.
- لا تستخدم كلمات طبية أو تشخيصية (مثل "اكتئاب"، "رهاب")، بل استخدم لغة يومية دافئة.
"""
        }
    ]

    # إضافة المحادثة السابقة
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})

    # إنشاء البرومبت
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1800).to(model.device)

    yield "Thinking... 🧠"

    # 🚀 التوليد الآمن
    output_ids = model.generate(
        **inputs,
        max_new_tokens=300,
        num_beams=3,
        temperature=0.7,
        top_p=0.9,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # استخراج الرد الجديد
    full_response = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    # تنظيف النص
    for tag in ["<|assistant|>", "<|user|>", "<|system|>", "assistant", "system", "user"]:
        full_response = full_response.replace(tag, "")
    full_response = full_response.strip()

    # بث تدريجي للرد
    response_text = ""
    for char in full_response:
        response_text += char
        time.sleep(0.01)
        yield response_text


# ======================================================
# 💬 واجهة Gradio
# ======================================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧘‍♀️ Arabic Mental Health Chatbot — Persistent Context + Smart Memory")
    chatbot = gr.Chatbot(label="💬 محادثتك", height=400)
    msg = gr.Textbox(placeholder="اكتب شعورك أو سؤالك هنا...", label="رسالتك")
    clear = gr.Button("🧹 مسح المحادثة")

    def user(user_message, history):
        history = history or []
        history.append((user_message, None))
        return "", history

    def bot(history):
        new_history = history.copy()
        user_message = new_history[-1][0]
        partial_text = ""

        for partial in generate_stream_response(new_history):
            partial_text = partial
            new_history[-1] = (user_message, partial_text)
            yield new_history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: [], None, chatbot)

demo.launch(share=True, debug=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipython-input-4249082201.py:99: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 محادثتك", height=400)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eb406898978ba5d7fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://eb406898978ba5d7fd.gradio.live


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, time, json, os

# 🔹 تحميل الموديل
model_path = "/content/drive/MyDrive/checkpoint-70"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# 📁 ملف لتخزين المحادثة
history_file = "/content/drive/MyDrive/chat_history.json"

# ======================================================
# 🧠 تحميل أو حفظ المحادثة
# ======================================================
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_history(history):
    with open(history_file, "w", encoding="utf-8") as f:
        json.dump(history, f, ensure_ascii=False, indent=2)

# ======================================================
# 🧠 توليد الرد (يحافظ على الذاكرة)
# ======================================================
def generate_stream_response(history):
    # حفظ آخر نسخة دائمًا
    save_history(history)

    # لو الحوار طويل، نختصره
    if len(history) > 10:
        summarized = "المحادثة السابقة كانت عن مشاعر المستخدم وبعض المخاوف النفسية، وكان الحديث داعمًا ومتفهمًا."
        history = [(summarized, None)] + history[-6:]

    # بناء المحادثة بنمط Qwen
    messages = [{
        "role": "system",
        "content": """
أنت مستشار نفسي رقمي ذكي متخصص في الدعم العاطفي والصحة النفسية.
تتكلم بلغة عربية بسيطة، هادئة، ومريحة.
استمع بتعاطف، وردّ بشكل مختصر (3–5 جمل) واقعي وداعم.
"""
    }]

    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1600).to(model.device)

    yield "Thinking... 🧠"

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=250,
            num_beams=2,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.05,
            no_repeat_ngram_size=3,
            early_stopping=True,
            max_time=30
        )

    full_response = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    # تنظيف
    for tag in ["<|assistant|>", "<|user|>", "<|system|>", "assistant", "system", "user"]:
        full_response = full_response.replace(tag, "")
    full_response = full_response.strip()

    # بث تدريجي
    response_text = ""
    for char in full_response:
        response_text += char
        time.sleep(0.01)
        yield response_text


# ======================================================
# 💬 واجهة Gradio مع ذاكرة حقيقية
# ======================================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧘‍♀️ Arabic Mental Health Chatbot — Memory Mode (يحفظ المحادثة)")

    chatbot = gr.Chatbot(label="💬 محادثتك", height=420)
    msg = gr.Textbox(placeholder="اكتب شعورك أو سؤالك هنا...", label="رسالتك")
    clear = gr.Button("🧹 مسح المحادثة")

    # تحميل آخر محادثة
    initial_history = load_history()

    def user(user_message, history):
        history = history or []
        history.append((user_message, None))
        save_history(history)
        return "", history

    def bot(history):
        new_history = history.copy()
        user_message = new_history[-1][0]
        for partial in generate_stream_response(new_history):
            new_history[-1] = (user_message, partial)
            yield new_history
        save_history(new_history)

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: ([], save_history([]))[0], None, chatbot)  # يمسح الملف والمحادثة

    chatbot.value = initial_history  # استرجاع آخر محادثة

demo.launch(share=True, debug=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipython-input-11678964.py:100: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💬 محادثتك", height=420)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a8b79ea6d8d225070.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8a8b79ea6d8d225070.gradio.live
